In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd

import rasterio
from rasterio.mask import mask
from rasterio.plot import show

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# === Load Algeria and Tunisia boundaries ===
world = gpd.read_file("datasets/soil/archive/ne_10m_admin_0_countries.shp")

if world.crs is None:
    world.set_crs(epsg=4326, inplace=True)  # WGS84

maghreb = world[world["ADMIN"].isin(["Algeria", "Tunisia"])]

# === Load the HWSD2 raster ===
raster_path = "datasets/soil/HWSD2_RASTER/HWSD2.bil"
with rasterio.open(raster_path) as src:
    # Reproject countries to match raster CRS
    maghreb = maghreb.to_crs(src.crs)
    
    # Clip raster with country polygons
    out_image, out_transform = mask(src, maghreb.geometry, crop=True)
    out_meta = src.meta.copy()

# === Update metadata for the clipped raster ===
out_meta.update({
    "driver": "GTiff",
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "transform": out_transform
})

# === Save the clipped raster ===
out_raster = "datasets/soil/HWSD2_Algeria_Tunisia.tif"
with rasterio.open(out_raster, "w", **out_meta) as dest:
    dest.write(out_image)


In [ ]:
import matplotlib.pyplot as plt
import rasterio
# Open the clipped GeoTIFF
raster_path = "datasets/soil/HWSD2_Algeria_Tunisia.tif"
with rasterio.open(raster_path) as src:
    print(src.crs)           # Check coordinate system
    print(src.shape)         # Height, width (rows, cols)
    print(src.count)         # Number of bands
    print(src.meta)          # Metadata summary
    
    # Read the first band (most rasters have just one)
    band1 = src.read(1)
    print(band1)
    
    valid = band1[band1 != src.nodata]
    
    unique_smu_algeria_tunisia = set(valid)
    
    print("valid", valid.shape, "\n", valid, "length unique smu", len(unique_smu_algeria_tunisia))
    
    # Display it
    plt.title("HWSD2 - Algeria and Tunisia")
    show(src)


In [ ]:
layers = pd.read_csv("datasets/soil/HWSD2_LAYERS_202510211033.csv")
print(layers.shape)
layers.head()

In [ ]:
layers_d1 = layers[layers["LAYER"] == "D1"]
print(layers_d1.shape)

In [ ]:
features = [
    "HWSD2_SMU_ID",
    "COARSE", "SAND", "SILT", "CLAY", "TEXTURE_USDA", "TEXTURE_SOTER",
    "BULK", "REF_BULK", "ORG_CARBON", "PH_WATER", "TOTAL_N", "CN_RATIO",
    "CEC_SOIL", "CEC_CLAY", "CEC_EFF", "TEB", "BSAT", "ALUM_SAT",
    "ESP", "TCARBON_EQ", "GYPSUM", "ELEC_COND", "SHARE"
]
print(layers_d1.columns)
layers_d1 = layers_d1[features]

layers_d1.shape


In [ ]:
layers_d1 = layers_d1.sort_values("SHARE", ascending=False)
dominant_layers = layers_d1.drop_duplicates("HWSD2_SMU_ID", keep="first")
print(dominant_layers.shape)
id_to_props = dominant_layers.set_index("HWSD2_SMU_ID").to_dict(orient="index")

In [ ]:
df = dominant_layers[dominant_layers['HWSD2_SMU_ID'].isin(unique_smu_algeria_tunisia)]
print(df.shape)
df.head()

In [ ]:
# ---- Define features you want to visualize ----
features = [
    "COARSE", "SAND", "SILT", "CLAY", "BULK", "REF_BULK",
    "PH_WATER", "TOTAL_N", "CN_RATIO", "CEC_SOIL", "CEC_CLAY",
    "CEC_EFF", "TEB", "BSAT", "ALUM_SAT", "ESP",
    "TCARBON_EQ", "GYPSUM", "ELEC_COND"
]

# ---- Create a dictionary of 2D maps (one per feature) ----
maps = {}

for feature in features:
    feature_map = np.full_like(band1, np.nan, dtype=float)

    for smu_id, props in id_to_props.items():
        value = props.get(feature)
        if pd.notna(value):
            feature_map[band1 == smu_id] = value

    maps[feature] = feature_map

# ---- Plot all maps in a grid ----
n_features = len(features)
n_cols = 4
n_rows = int(np.ceil(n_features / n_cols))

fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, n_rows * 4))
axes = axes.flatten()

for i, feature in enumerate(features):
    ax = axes[i]
    im = ax.imshow(maps[feature], cmap="viridis")
    ax.set_title(feature, fontsize=10)
    ax.axis("off")
    plt.colorbar(im, ax=ax, shrink=0.6)

# Hide unused subplots if any
for j in range(i + 1, len(axes)):
    axes[j].axis("off")

plt.suptitle("Soil Properties (Top 20 cm — HWSD2)", fontsize=16)
plt.tight_layout()
plt.show()


# Univariant analysis

## numerical features

In [ ]:
numeric_df = df.select_dtypes(include='number')

In [ ]:
num_cols = [
    "COARSE", "SAND", "SILT", "CLAY", "BULK", "REF_BULK",
    "ORG_CARBON", "PH_WATER", "TOTAL_N", "CN_RATIO",
    "CEC_SOIL", "CEC_CLAY", "CEC_EFF", "TEB", "BSAT",
    "ALUM_SAT", "ESP", "TCARBON_EQ", "GYPSUM", "ELEC_COND"
]

In [ ]:
import math

# Define grid dimensions (e.g., 10 rows of 4 plots each)
cols_per_row = 4 
total_plots = len(num_cols) * 2
rows = math.ceil(total_plots / cols_per_row)

fig, axes = plt.subplots(rows, cols_per_row, figsize=(20, 5 * rows))
axes = axes.flatten() # Flatten to iterate easily

for i, col in enumerate(num_cols):
    # Plot Histogram at index 2*i
    sns.histplot(df[col], kde=True, ax=axes[2*i], color='skyblue')
    axes[2*i].set_title(f"{col} Hist")
    
    # Plot Boxplot at index 2*i + 1
    sns.boxplot(x=df[col], ax=axes[2*i + 1], color='lightcoral')
    axes[2*i + 1].set_title(f"{col} Box")

# Remove any empty subplots at the end
for j in range(total_plots, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

## categorical features

In [ ]:
# Set up a single figure with 1 row and 2 columns
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

cat_cols = ["TEXTURE_USDA", "TEXTURE_SOTER"]


for i, col in enumerate(cat_cols):
    # Plot on the specific axis (axes[0] or axes[1])
    sns.countplot(
        x=df[col], 
        order=df[col].value_counts().index, 
        ax=axes[i], 
        palette="viridis"
    )
    
    axes[i].set_title(f"Distribution of {col}", fontsize=14)
    axes[i].tick_params(axis='x', rotation=45)
    
    # Print the percentage distribution below the plots
    print(f"\n--- Percentage Distribution for {col} ---")
    print(df[col].value_counts(normalize=True) * 100)

plt.tight_layout()
plt.show()

## Summary 

In [ ]:
df[num_cols].describe().T # mode

In [ ]:
df.mode().iloc[0].T

# Bivariate analysis

## top correlated pairs

In [ ]:
corr = numeric_df.corr()

corr_pairs = corr.unstack().sort_values(kind="quicksort", ascending=False)
corr_pairs = corr_pairs[corr_pairs < 1]  # remove self-correlation
top_pairs = corr_pairs[abs(corr_pairs) > 0.6]  # focus on strong correlations

print(top_pairs.head(10))

In [ ]:
sns.pairplot(df, vars=top_pairs.index.get_level_values(0).unique())
plt.show()

# Multivariate analysis

In [ ]:
corr = numeric_df.corr()
plt.figure(figsize=(14, 10))
sns.heatmap(corr[(corr >= 0.5) | (corr <= -0.5)], annot=True, cmap="coolwarm", center=0)
plt.title("Strong Correlations (|r| ≥ 0.5)")
plt.show()